In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.spatial.distance as dist
%matplotlib inline
import pylab
import json

from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_users = pd.read_json('D:/desk_2104/CS Sem4/CS6220/Project/dataset/user.json', lines=True)

In [3]:
df_users.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,3.80,5174,284,5174,5175,78,299,1435,7829,7397,...,16856,"[2014, 2016, 2013, 2011, 2012, 2015, 2010, 2017]",209,"[M19NwFwAXKRZzt8koF11hQ, QRcMZ8pJJBBZaKubHOoMD...",16605,Cin,272,17019,lsSiIjAKVl-QRxKjRErBeg,2010-07-13
1,3.94,1556,211,1556,1285,101,134,1295,162,2134,...,40110,"[2014, 2017, 2011, 2012, 2015, 2009, 2013, 200...",835,"[eoSSJzdprj3jxXyi94vDXg, QF0urZa-0bxga17ZeY-9l...",10882,Andrea,2559,83681,om5ZiponkpRqUNa3pVPiRg,2006-01-18
2,4.72,15,1,15,5,0,1,11,8,20,...,55,[],17,"[Oa84FFGBw1axX8O6uDkmqg, SRcWERSl4rhm-Bz9zN_J8...",4,Gabe,277,45,-lGwMGHMC_XihFJNKCJNRg,2014-10-31
3,3.76,9,0,9,1,0,1,4,1,11,...,4,"[2016, 2017]",11,"[96DJovjKAtExnyBZRfniPQ, Djczq-bRn761U4FW77NUt...",4,Leah,436,15,D-ydMTPGWXTVm4_jjp0k9g,2013-04-01
4,4.23,276,0,276,59,0,8,51,169,386,...,6006,"[2017, 2016]",49,"[iN0A6QIrEFYoSGHFaknh8Q, B2HDoWNIzLlon0IhS1cmD...",360,Juan,921,9152,PcvbBOCOcs6_suRDH7TSTg,2012-08-16


In [4]:
#Reading big file line by line, taking first 1000 reviews
c=0
s = open('D:/desk_2104/CS Sem4/CS6220/Project/dataset/review_sub.json','w',encoding='utf8')
with open('D:/desk_2104/CS Sem4/CS6220/Project/dataset/review.json', encoding='utf8') as f:
    for line in f:
        c=c+1
        s.write(line)
        if(c == 10000):
            break

In [5]:
df_reviews = pd.read_json('D:/desk_2104/CS Sem4/CS6220/Project/dataset/review_sub.json', lines=True)

In [6]:
df_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


In [53]:
df_bus = pd.read_json('D:/desk_2104/CS Sem4/CS6220/Project/dataset/business.json', lines=True)

In [54]:
res_mer = pd.merge(df_bus,df_reviews, on='business_id')

In [55]:
res_mer.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,stars_x,state,cool,date,funny,review_id,stars_y,text,useful,user_id
0,350 E Steels Corners Rd,"{'Alcohol': 'beer_and_wine', 'HasTV': True, 'N...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Monday': '11:00-21:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,0,2013-08-30,0,yopVrikncv_54eR44R9sHQ,4,Great variety and the best dog in town. Try th...,0,Q9nZLjnQ_0RiWfNjCRt9sQ
1,350 E Steels Corners Rd,"{'Alcohol': 'beer_and_wine', 'HasTV': True, 'N...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Monday': '11:00-21:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,0,2013-12-09,0,fQn5gA-jXYHefi2Kxx9rcg,1,This was my first visit and I was definitely N...,1,9Vu2eIk0HXGAD2h1Vr81RA
2,350 E Steels Corners Rd,"{'Alcohol': 'beer_and_wine', 'HasTV': True, 'N...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Monday': '11:00-21:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,3,2012-06-19,3,HzreYQD3_VOa-MvleCQhEw,4,Went to their grand opening on the June 17th a...,6,MBf09zdFcOEnYA84aENzXA
3,350 E Steels Corners Rd,"{'Alcohol': 'beer_and_wine', 'HasTV': True, 'N...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Monday': '11:00-21:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,0,2015-10-27,0,E73McNvR0dT-l-7mlsh_7A,3,Good food. Way over priced as everything is ex...,2,cA7hAhlsXIotA4wRLdAZBQ
4,350 E Steels Corners Rd,"{'Alcohol': 'beer_and_wine', 'HasTV': True, 'N...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Monday': '11:00-21:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,0,2015-11-18,0,6D0JvHSBurLYbC_o_drcJA,4,This is my regular lunch stop when I am in Sto...,0,UCnk2nOq6cvtjboMWkhO9A


In [16]:
df = res_mer.pivot(index = 'user_id', columns ='business_id', values='stars_y')
df.head()

business_id,-MhfebM0QIsKt87iDN-FNw,-cYOKJ5kbVZqzSYQIzZcqA,-iC6giVhI7vY6W_dnw08YA,-pV9kWNoA9vyHfM_auYecA,03SYJLErY8XpNfY-qiDZcw,0nyM_mub7LxN6Dx7tYpMaA,1pXyaPUXMqZyxW1Ew7BLUg,2L4rNP5kjRXCxxspYPRTdA,2UmvDC88Wj3eKOBLFTh86w,2Z2QuLYY0FvUeJ75ane-Zw,...,wzuhuTptNFTi4MZkrCItBA,xOx-N9G7AteZTnZfVFfNHA,xjViuMrrApPM6Rnz8ovOVw,xpW7Qp8JpYys0IGG0ENZaw,y7yo-LAjJnNoYwWX6ZYbqA,yRk8ezAFV59HDHsuHiiU6Q,ykSlMTRgyWzAmhP7ywc9gg,ypuo8mzg0uCxl24K08rIig,zoq5SERVEDBd7MaKPjlLJA,zw4Legbcu018p5WcZ74iWA
user_id,,,,,,,,,,,,,,,,,,,,,
---1lKK3aKOuomHnwAkAow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Nnm_506G_p8MxAOQna5w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--P-Qvza7AED8gnDrZkMgA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ZNfWKj1VyVElRx6-g1fg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-00MbjbaOISrcuV7jOVRIg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
items=df.columns.values

In [63]:
mat = df.as_matrix().astype(np.float)

In [64]:
# centralizing to mean zero
sum =0.0
count =0
for i in range(len(mat)):
    sum = 0.0
    count =0
    for j in range(len(mat[0])):
        if(math.isnan(mat[i][j])):
            continue
        sum += mat[i][j]
        count += 1
    avg = sum/count
    for j in range(len(mat[0])):
        if(math.isnan(mat[i][j])):
            continue
        mat[i][j] -= avg

In [65]:
ind = np.where(np.isnan(mat))
mat[ind] = 0

In [ ]:
df

In [66]:
#creating sparse matrix
sp_mat = sp.csr_matrix(mat)

In [67]:
print("Number of users:", len(mat))
print("Number of items:", len(mat[0]))

Number of users: 9578
Number of items: 227


In [68]:
i = 0
for i in range(len(mat)):
    c = 0
    for j in range(len(mat[0])):
        if mat[i][j] > 0:
            c = c+1
    if c >4:
        break
print(i)

3935


In [87]:
# List of indices of similar users
similar_users = []
def cosine_similarity(user):
    for i in range(len(mat)):
        cs = 1 - dist.cosine(mat[i],user)
        if cs >= 0.5:
            similar_users.append(i)

In [88]:
#calling method
#Let new user be mat[3935]
cosine_similarity(mat[3935])

C:\Users\Gautam Vashisht\Anaconda3\lib\site-packages\scipy\spatial\distance.py:326: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [89]:
len(similar_users)

32

In [90]:
###Item Recommendation to users based on similar_users####